#Data Analysis in Terminal

##Quality Control

### _Quaulity Check With Fastqc_

To get help,

In [1]:
fastqc --help


            FastQC - A high throughput sequence QC analysis tool

SYNOPSIS

	fastqc seqfile1 seqfile2 .. seqfileN

    fastqc [-o output dir] [--(no)extract] [-f fastq|bam|sam] 
           [-c contaminant file] seqfile1 .. seqfileN

DESCRIPTION

    FastQC reads a set of sequence files and produces from each one a quality
    control report consisting of a number of different modules, each one of 
    which will help to identify a different potential type of problem in your
    data.
    
    If no files to process are specified on the command line then the program
    will start as an interactive graphical application.  If files are provided
    on the command line then the program will run with no user interaction
    required.  In this mode it is suitable for inclusion into a standardised
    analysis pipeline.
    
    The options for the program as as follows:
    
    -h --help       Print this help file and exit
    
    -v --version    Print the version of the program and exit

In [ ]:
cd RNA-Seq/QC/Fastqc_Out

In [ ]:
fastqc -t 4  RNA-Seq/RAW_Data/3290-TM-0001-18_S18_L002_R1_001-2.fastq.gz \ 
             RNA-Seq/RAW_Data/3290-TM-0001-18_S18_L004_R1_001-2.fastq.gz

### Code For Many Samples
If your raw data path names end with _ *.fastq_ change the _ *.fastq.gz _  to  _.fastq_ in the following code.

In [ ]:
for f in RNA-Seq/RAW_Data/*.fastq.gz; do fastqc -t 4 $f  ; done

### _Adapter Trimming with scythe _

In [1]:
scythe --help


Usage: scythe -a adapter_file.fasta sequence_file.fastq
Trim 3'-end adapter contaminants off sequence files. If no output file
is specified, scythe will use stdout.

Options:
  -p, --prior		prior (default: 0.300)
  -q, --quality-type	quality type, either illumina, solexa, or sanger (default: sanger)
  -m, --matches-file	matches file (default: no output)
  -o, --output-file	output trimmed sequences file (default: stdout)
  -t, --tag		add a tag to the header indicating Scythe cut a sequence (default: off)
  -n, --min-match	smallest contaminant to consider (default: 5)
  -M, --min-keep	filter sequnces less than or equal to this length (default: 35)
  --quiet		don't output statistics about trimming to stdout (default: off)
  --help		display this help and exit
  --version		output version information and exit

  Information on quality schemes:
  phred			PHRED quality scores (e.g. from Roche 454). ASCII with no offset, range: [4, 60].
  sanger		Sanger are PHRED ASCII qualities with an offset

Unzip your data before this step,

In [ ]:
gzip -d RNA-Seq/RAW_Data/3290-TM-0001-18_S18_L002_R1_001-2.fastq.gz
gzip -d RNA-Seq/RAW_Data/3290-TM-0001-18_S18_L004_R1_001-2.fastq.gz

### gzip -d Code For Many Samples

In [ ]:
for f in RNA-Seq/RAW_Data/*.gz; do gzip -d  $f  ; done

In [ ]:
scythe  -a RNA-Seq/Adaptors/TruSeq_adapters.fasta  -M 50 \
        -o RNA-Seq/QC/Adapter_Removed/Adapt_rem_3290-TM-0001-18_S18_L002_R1_001-2.fastq \
                                             RNA-Seq/RAW_Data/3290-TM-0001-18_S18_L002_R1_001-2.fastq

<img src="images/SC.png">

In [ ]:
scythe  -a RNA-Seq/Adaptors/TruSeq_adapters.fasta  -M 50 -o RNA-Seq/QC/Adapter_Removed/Adapt_rem_3290-TM-0001-18_S18_L004_R1_001-2.fastq \
                                             RNA-Seq/RAW_Data//3290-TM-0001-18_S18_L004_R1_001-2.fastq

### Scythe Code For Many Samples

In [ ]:
for f in RNA-Seq/RAW_Data/*.fastq; scythe -a RNA-Seq/Adaptors/TruSeq_adapters.fasta \ 
-o RNA-Seq/QC/Adapter_Removed/Adapt_rem_${f#*/}   $f  ; done

### _Quality Trimming with sickle _

In [2]:
sickle se --help


Usage: sickle se [options] -f <fastq sequence file> -t <quality type> -o <trimmed fastq file>

Options:
-f, --fastq-file, Input fastq file (required)
-t, --qual-type, Type of quality values (solexa (CASAVA < 1.3), illumina (CASAVA 1.3 to 1.7), sanger (which is CASAVA >= 1.8)) (required)
-o, --output-file, Output trimmed fastq file (required)
-q, --qual-threshold, Threshold for trimming based on average quality in a window. Default 20.
-l, --length-threshold, Threshold to keep a read based on length after trimming. Default 20.
-x, --no-fiveprime, Don't do five prime trimming.
-n, --trunc-n, Truncate sequences at position of first N.
-g, --gzip-output, Output gzipped files.
--quiet, Don't print out any trimming information
--help, display this help and exit
--version, output version information and exit



In [ ]:
sickle se -q 20  -t sanger -f RNA-Seq/QC/Adapter_Removed/Adapt_rem_3290-TM-0001-18_S18_L002_R1_001-2.fastq  \
                             -o RNA-Seq/QC/Trimmed/Q_trimmed_3290-TM-0001-18_S18_L002_R1_001-2.fastq

<img src="images/SIC.png">

### Sickle Code For Many Samples


In [ ]:
for f in RNA-Seq/QC/Adapter_Removed/*.fastq; sickle se -q 20  -t sanger  -f $f   \
-o RNA-Seq/QC/Trimmed/Q_trimmed_${f#*/} ; done

### _Short read aligning with Tophat2 _

* Make bowtie2 indexes for your Genome

In [ ]:
cd RNA-Seq/Reference/Genome/

In [ ]:
gzip -d Gmax_275_v2.0.gz

In [ ]:
mv Gmax_275_v2.0 Gmax_275_v2.0.fa

In [ ]:
bowtie2-build Gmax_275_v2.0.fa Gmax_275_v2.0


> WARNING: THIS WILL TAKE LONG TIME

* Align short reads to Genome using Tophat2

In [ ]:
cd ~/RNA-Seq

In [ ]:
tophat2 --num-threads 4  --output-dir RNA-Seq/Alignment/Tophat2 RNA-Seq/Reference/Genome/Gmax_275_v2.0 \
                                RNA-Seq/QC/Trimmed/Q_trimmed_3290-TM-0001-18_S18_L002_R1_001-2.fastq

### Tophat2 Code For Many Samples

In [ ]:
for f in RNA-Seq/QC/Trimmed/*.fastq; tophat2 --num-threads 4  --output-dir RNA-Seq/Alignment/Tophat2 -f $f   \
RNA-Seq/Reference/Genome/Gmax_275_v2.0  $f; done

#Excercise

1. Run Cufflinks2 on alignment file(SAM)